# Tutorial Goal

This tutorial aims to show some example of **trace analysis and visualization**
using a pre-defined set of analysis and plotting functions provided by the
**Filters** and **TraceAnalysis** modules of LISA.

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')
# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)

In [2]:
# Generate plots inline
%pylab inline

# Python modules required by this notebook
import json
import os

Populating the interactive namespace from numpy and matplotlib


# Configuration

In [16]:
# Let's use an example trace
res_dir = '/data/work/juno/lisa/results/ExecutorExample/rtapp:eas:rta/1'
tracefile = os.path.join(res_dir, 'trace.txt')
platformfile = os.path.join(res_dir, '../platform.json')
!tree {res_dir}

/data/work/juno/lisa/results/ExecutorExample/rtapp:eas:rta
├── 1
│   ├── energy.json
│   ├── output.log
│   ├── rta_00.json
│   ├── rt-app-task_p20-0.log
│   ├── trace.dat
│   └── trace.txt
├── kernel.config
├── kernel.version
└── platform.json

1 directory, 9 files


In [17]:
# Trace events of interest
events_to_parse = [
    "sched_switch",
    "sched_wakeup",
    "sched_wakeup_new",
    "sched_contrib_scale_f",
    "sched_load_avg_cpu",
    "sched_load_avg_task",
    "sched_tune_config",
    "sched_tune_tasks_update",
    "sched_tune_boostgroup_update",
    "sched_tune_filter",
    "sched_boost_cpu",
    "sched_boost_task",
    "sched_energy_diff",
    "cpu_frequency",
    "cpu_capacity",
]

# Platform description
with open(platformfile, 'r') as fh:
    platform = json.load(fh)

logging.info("CPUs max capacities:")
logging.info("   big: %5d (cpus: %s)",
             platform['nrg_model']['big']['cpu']['cap_max'],
             platform['clusters']['big'])
logging.info("LITTLE: %5d (cpus: %s)",
             platform['nrg_model']['little']['cpu']['cap_max'],
             platform['clusters']['little'])

# Time range from the analysis
(t_min, t_max) = (0, None)

04:39:02  INFO    : CPUs max capacities:
04:39:02  INFO    :    big:  1024 (cpus: [1, 2])
04:39:02  INFO    : LITTLE:   593 (cpus: [0, 3, 4, 5])


# Trace parsing

In [18]:
# Load the LISA::Trace parsing module
from trace import Trace


# The LISA::Trace module is a wrapper of the TRAPpy FTrace module which
# allows to keep track of platform specific details to support the generation
# of
trace = Trace(platform, res_dir, events_to_parse, window=(t_min,t_max))

04:39:06  INFO    : Parsing FTrace format...


IOError: [Errno 2] No such file or directory: '/data/work/juno/lisa/results/ExecutorExample/rtapp:eas:rta/trace.txt'

Notice how some platform specific data are collected and reported by the
LISA::Trace module

In [ ]:
# This is the standard TRAPpy::FTrace object, already configured for the
# analysis related to the events of interest
ftrace = trace.ftrace
logging.info("List of events identified in the trace:")
for event in ftrace.class_definitions.keys():
    logging.info("   %s", event)

In [ ]:
# Original TRAPpy::FTrace DataSet are still accessible by specifying the
# trace event name of interest
trace.df('sched_load_avg_task').head()

# Predefined LISA Filtering Functions

In [ ]:
from filters import Filters

fl = Filters(trace)
fl.setXTimeRange(t_min, t_max)

## Top BIG tasks

In [ ]:
# Get a list of tasks which are the most big in the trace
top_big_tasks = fl.topBigTasks(
    max_tasks=3,          # Maximum number of tasks to report
    min_utilization=None, # Minimum utilization to be considered "big"
                          # default: LITTLE CPUs max capacity
    min_samples=100,      # Number of samples over the minimum utilization 
)

In [ ]:
# The collected information is available for further analysis
top_big_tasks

In [ ]:
# Plot utilization of "big" tasks decorated with platform specific capacity information
fl.plotBigTasks()

## Top WAKEUP tasks

In [ ]:
top_wakeup_tasks = fl.topWakeupTasks(
    max_tasks=5,     # Maximum number of tasks to report
    min_wakeups=100  # Minimum number of wakeup to be reported
)

### Wakeup vs Forks

In [ ]:
fl.plotWakeupTasks(per_cluster=False)

### Wakeup per cluster

In [ ]:
fl.plotWakeupTasks(per_cluster=True)

## RT Tasks

In [ ]:
fl.rtTasks(max_prio=100)

# Predefined LISA TraceAnslysis Functions

In [ ]:
# Import the LISA::TraceAnalysis module
from trace_analysis import TraceAnalysis

ta = TraceAnalysis(
    trace,               # LISA::Trace object
    tasks=top_big_tasks, # (optional) list of tasks to plot
    plotsdir=res_dir
)

In [ ]:
# Define time ranges for all the time based plots
ta.setXTimeRange(t_min, t_max)

In [ ]:
ta.plotTasks(top_big_tasks)

In [ ]:
# Cluster frequencies
ta.plotClusterFrequencies()

In [ ]:
# Plots SchedTune's Energy-Diff Space Filtering
ta.plotEDiffTime(top_big_tasks)